In [2]:
import pandas as pd
from yahoo_fin import stock_info as si
from datetime import datetime, timedelta
import joblib
import plotly.graph_objects as go

def calculate_technical_indicators(data):
    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    data['RSI_14'] = 100 - (100 / (1 + rs))
    
    data['20_SMA'] = data['Close'].rolling(window=20).mean()
    data['20_std'] = data['Close'].rolling(window=20).std()
    data['Upper_Band'] = data['20_SMA'] + (data['20_std'] * 2)
    data['Lower_Band'] = data['20_SMA'] - (data['20_std'] * 2)
    
    data['Close_1'] = data['Close'].shift(1)
    data['Close_2'] = data['Close'].shift(2)
    
    data.dropna(inplace=True)
    return data

def fetch_historical_data(ticker, start_date):
    try:
        end_date = datetime.today().strftime('%Y-%m-%d')
        data = si.get_data(ticker, start_date=start_date, end_date=end_date)
        data = data.reset_index()

        if 'close' not in data.columns:
            raise ValueError("'Close' column not found in the data")

        data = data[['index', 'close', 'open', 'high', 'low', 'volume']]  # Ensure only needed columns are included
        data.rename(columns={'index': 'Date', 'close': 'Close', 'open': 'Open', 'high': 'High', 'low': 'Low', 'volume': 'Volume'}, inplace=True)
        
        data = calculate_technical_indicators(data)
        return data

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

def display_forecasts_from_data(df, ticker, periods=30):
    if df.empty:
        print(f"Error: Empty DataFrame for {ticker}")
        return

    df.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)
    df.dropna(inplace=True)

    # Load the saved model
    model_filename = f"Models/{ticker}_prophet_model.pkl"
    try:
        model = joblib.load(model_filename)
    except Exception as e:
        print(f"Error loading model for {ticker}: {e}")
        return

    last_date = df['ds'].iloc[-1]
    future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=periods, freq='D')
    future = pd.DataFrame({'ds': future_dates})
    
    # Add regressors to future dataframe
    future['SMA_10'] = df['SMA_10'].iloc[-1]
    future['RSI_14'] = df['RSI_14'].iloc[-1]
    future['Upper_Band'] = df['Upper_Band'].iloc[-1]
    future['Lower_Band'] = df['Lower_Band'].iloc[-1]
    future['Close_1'] = df['y'].iloc[-1]
    future['Close_2'] = df['y'].iloc[-2]

    forecast = model.predict(future)

    #print(forecast[['ds', 'yhat']])
    forecast['Forecasted Close'] = df['y'].iloc[-1] + (forecast['yhat'] - forecast['yhat'].iloc[0])
    forecast['Forecasted High'] = df['y'].iloc[-1] + (forecast['yhat_upper'] - forecast['yhat_upper'].iloc[0])
    forecast['Forecasted Low'] = df['y'].iloc[-1] + (forecast['yhat_lower'] - forecast['yhat_lower'].iloc[0])
    
    # Generate candlestick data
    forecast['Open'] = forecast['Forecasted Close'].shift(1)
    # forecast['High'] = forecast[['Forecasted Close', 'Open']].max(axis=1)
    # forecast['Low'] = forecast[['Forecasted Close', 'Open']].min(axis=1)
    forecast['High'] = forecast['Forecasted High']
    forecast['Low'] = forecast['Forecasted Low']
    forecast['Close'] = forecast['Forecasted Close']
    forecast.dropna(inplace=True)

    fig = go.Figure(data=[go.Candlestick(x=forecast['ds'],
                                         open=forecast['Open'],
                                         high=forecast['High'],
                                         low=forecast['Low'],
                                         close=forecast['Close'],
                                         increasing_line_color='green', decreasing_line_color='red')])
    

     # Adjust y-axis range for a tighter view
    y_min = forecast['Low'].min() * 0.990
    y_max = forecast['High'].max() * 1.007

    fig.update_layout(title=f'{ticker} Forecast Candlestick Chart',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      yaxis_range=[y_min, y_max],
                      xaxis_rangeslider_visible=False, xaxis=dict(tickmode='linear'))

    fig.show()

def process_multiple_tickers(tickers, start_date):
    for ticker in tickers:
        historical_data = fetch_historical_data(ticker, start_date)
        if not historical_data.empty:
            display_forecasts_from_data(historical_data, ticker)

# Example usage
tickers = ["AAPL", "ABBV", "ADBE", "AMZN", "AVGO", "BRK-B", "CRM", "COST", "CVX", "HD", 
          "JNJ", "JPM", "LLY", "MA", "META", "MRK", "MSFT", "NVDA", "PG", "TSLA", "UNH", "V", "XOM"]  # Array of tickers
start_date = datetime.today() - timedelta(days=730)  # Approximately 2 years ago

process_multiple_tickers(tickers, start_date)


In [19]:
import pandas as pd
from yahoo_fin import stock_info as si
from datetime import datetime, timedelta
import joblib
import plotly.express as px

def calculate_technical_indicators(data):
    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    data['RSI_14'] = 100 - (100 / (1 + rs))
    
    data['20_SMA'] = data['Close'].rolling(window=20).mean()
    data['20_std'] = data['Close'].rolling(window=20).std()
    data['Upper_Band'] = data['20_SMA'] + (data['20_std'] * 2)
    data['Lower_Band'] = data['20_SMA'] - (data['20_std'] * 2)
    
    data['Close_1'] = data['Close'].shift(1)
    data['Close_2'] = data['Close'].shift(2)
    
    data.dropna(inplace=True)
    return data

def fetch_historical_data(ticker, start_date):
    try:
        end_date = datetime.today().strftime('%Y-%m-%d')
        data = si.get_data(ticker, start_date=start_date, end_date=end_date)
        data = data.reset_index()

        # Debugging: Inspect the fetched data structure
        print(f"Fetched data for {ticker}:\n", data.head())
        
        if 'close' not in data.columns:
            raise ValueError("'Close' column not found in the data")

        data = data[['index', 'close']]  # Ensure only needed columns are included
        data.rename(columns={'index': 'Date', 'close': 'Close'}, inplace=True)
        
        data = calculate_technical_indicators(data)
        return data

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

def display_forecasts_from_data(df, ticker, periods=30):
    if df.empty:
        print(f"Error: Empty DataFrame for {ticker}")
        return

    df.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)
    df.dropna(inplace=True)

    # Load the saved model
    model_filename = f"Models/{ticker}_prophet_model.pkl"
    try:
        model = joblib.load(model_filename)
    except Exception as e:
        print(f"Error loading model for {ticker}: {e}")
        return

    last_date = df['ds'].iloc[-1]
    future_dates = pd.date_range(start=last_date + timedelta(days=1), periods=periods, freq='D')
    future = pd.DataFrame({'ds': future_dates})
    
    # Add regressors to future dataframe
    future['SMA_10'] = df['SMA_10'].iloc[-1]
    future['RSI_14'] = df['RSI_14'].iloc[-1]
    future['Upper_Band'] = df['Upper_Band'].iloc[-1]
    future['Lower_Band'] = df['Lower_Band'].iloc[-1]
    future['Close_1'] = df['y'].iloc[-1]
    future['Close_2'] = df['y'].iloc[-2]

    forecast = model.predict(future)
    
    # Ensure Forecasted Close Price is calculated correctly
    forecast['Forecasted Close'] = df['y'].iloc[-1] + (forecast['yhat'] - forecast['yhat'].iloc[0])

    print(f"Forecasted Close Prices for {ticker}:")
    print(forecast[['ds', 'Forecasted Close']])

    fig = px.line(forecast, x='ds', y='Forecasted Close', labels={'Forecasted Close': 'Forecast'}, title=f'{ticker} Forecast')
    fig.show()

def process_multiple_tickers(tickers, start_date):
    for ticker in tickers:
        historical_data = fetch_historical_data(ticker, start_date)
        if not historical_data.empty:
            display_forecasts_from_data(historical_data, ticker)

# Example usage
tickers = ["AAPL", "ABBV", "ADBE", "AMZN", "AVGO", "BRK-B", "CRM", "COST", "CVX", "HD", 
          "JNJ", "JPM", "LLY", "MA", "META", "MRK", "MSFT", "NVDA", "PG", "TSLA", "UNH", "V", "XOM"]  # Array of tickers
start_date = datetime.today() - timedelta(days=730)  # Approximately 2 years ago

process_multiple_tickers(tickers, start_date)



Fetched data for AAPL:
        index        open        high         low       close    adjclose  \
0 2022-05-24  140.809998  141.970001  137.330002  140.360001  138.789780   
1 2022-05-25  138.429993  141.789993  138.339996  140.520004  138.947998   
2 2022-05-26  137.389999  144.339996  137.139999  143.779999  142.171524   
3 2022-05-27  145.389999  149.679993  145.259995  149.639999  147.965958   
4 2022-05-31  149.070007  150.660004  146.839996  148.839996  147.174896   

      volume ticker  
0  104132700   AAPL  
1   92482700   AAPL  
2   90601500   AAPL  
3   90978500   AAPL  
4  103718400   AAPL  
Forecasted Close Prices for AAPL:
           ds  Forecasted Close
0  2024-05-23        190.899994
1  2024-05-24        191.060191
2  2024-05-25        191.945487
3  2024-05-26        191.951186
4  2024-05-27        190.900053
5  2024-05-28        191.091821
6  2024-05-29        191.046096
7  2024-05-30        190.939886
8  2024-05-31        191.100084
9  2024-06-01        191.985379
1

Fetched data for ABBV:
        index        open        high         low       close    adjclose  \
0 2022-05-24  147.029999  151.350006  146.610001  149.110001  137.943207   
1 2022-05-25  150.360001  152.990005  149.789993  151.960007  140.579758   
2 2022-05-26  151.630005  153.070007  150.389999  150.570007  139.293854   
3 2022-05-27  149.520004  150.050003  147.750000  150.000000  138.766541   
4 2022-05-31  148.639999  148.880005  146.669998  147.369995  136.333466   

     volume ticker  
0   6367200   ABBV  
1   5042600   ABBV  
2   6659300   ABBV  
3   8405000   ABBV  
4  11227800   ABBV  
Forecasted Close Prices for ABBV:
           ds  Forecasted Close
0  2024-05-23        159.610001
1  2024-05-24        159.621725
2  2024-05-25        159.088490
3  2024-05-26        159.091134
4  2024-05-27        159.574455
5  2024-05-28        159.561483
6  2024-05-29        159.562606
7  2024-05-30        159.628508
8  2024-05-31        159.640232
9  2024-06-01        159.106997
10 2024

Fetched data for ADBE:
        index        open        high         low       close    adjclose  \
0 2022-05-24  401.609985  404.329987  391.040009  398.410004  398.410004   
1 2022-05-25  395.320007  406.100006  394.709991  402.500000  402.500000   
2 2022-05-26  405.089996  411.549988  402.459991  408.600006  408.600006   
3 2022-05-27  414.799988  428.549988  414.630005  428.220001  428.220001   
4 2022-05-31  427.220001  427.290009  412.790009  416.480011  416.480011   

    volume ticker  
0  2630800   ADBE  
1  2726700   ADBE  
2  2419500   ADBE  
3  3560400   ADBE  
4  7944200   ADBE  
Forecasted Close Prices for ADBE:
           ds  Forecasted Close
0  2024-05-23        483.929993
1  2024-05-24        483.384206
2  2024-05-25        514.063326
3  2024-05-26        514.061404
4  2024-05-27        483.525188
5  2024-05-28        483.559486
6  2024-05-29        483.014252
7  2024-05-30        483.916698
8  2024-05-31        483.370912
9  2024-06-01        514.050031
10 2024-06-02

Fetched data for AMZN:
        index        open        high         low       close    adjclose  \
0 2022-05-24  104.025002  105.400002  101.260002  104.099998  104.099998   
1 2022-05-25  103.655502  108.175003  103.650002  106.775002  106.775002   
2 2022-05-26  107.970001  112.665497  107.453499  111.077499  111.077499   
3 2022-05-27  113.550003  115.186996  112.627998  115.146500  115.146500   
4 2022-05-31  116.279999  121.994499  115.675003  120.209503  120.209503   

      volume ticker  
0  102934000   AMZN  
1   93120000   AMZN  
2   93002000   AMZN  
3   93660000   AMZN  
4  144634000   AMZN  
Forecasted Close Prices for AMZN:
           ds  Forecasted Close
0  2024-05-23        183.130005
1  2024-05-24        182.690578
2  2024-05-25        194.892716
3  2024-05-26        194.889411
4  2024-05-27        182.550096
5  2024-05-28        182.710018
6  2024-05-29        182.910127
7  2024-05-30        183.106885
8  2024-05-31        182.667458
9  2024-06-01        194.869596
1

Fetched data for AVGO:
        index        open        high         low       close    adjclose  \
0 2022-05-24  526.659973  528.869995  516.190002  524.200012  497.802643   
1 2022-05-25  523.900024  537.450012  523.900024  531.630005  504.858490   
2 2022-05-26  531.539978  554.570007  527.719971  550.659973  522.930115   
3 2022-05-27  562.090027  585.460022  560.010010  583.280029  553.907471   
4 2022-05-31  584.500000  587.030029  576.000000  580.130005  550.916138   

    volume ticker  
0  2506000   AVGO  
1  1784200   AVGO  
2  3974300   AVGO  
3  3730100   AVGO  
4  3000900   AVGO  
Forecasted Close Prices for AVGO:
           ds  Forecasted Close
0  2024-05-23       1392.239990
1  2024-05-24       1391.888857
2  2024-05-25       1377.730203
3  2024-05-26       1377.721758
4  2024-05-27       1391.988119
5  2024-05-28       1392.053549
6  2024-05-29       1391.140852
7  2024-05-30       1392.180876
8  2024-05-31       1391.829743
9  2024-06-01       1377.671089
10 2024-06-02

...

Fetched data for PG:
        index        open        high         low       close    adjclose  \
0 2022-05-24  144.889999  147.940002  144.820007  147.630005  140.241348   
1 2022-05-25  147.110001  147.139999  143.500000  145.210007  137.942490   
2 2022-05-26  146.479996  147.380005  145.039993  146.479996  139.148911   
3 2022-05-27  146.970001  148.720001  146.630005  148.720001  141.276825   
4 2022-05-31  147.289993  148.990005  146.160004  147.880005  140.478851   

     volume ticker  
0   7207400     PG  
1   8217800     PG  
2   5702600     PG  
3   6028800     PG  
4  12416700     PG  
Forecasted Close Prices for PG:
           ds  Forecasted Close
0  2024-05-23        167.809998
1  2024-05-24        168.240703
2  2024-05-25        169.162270
3  2024-05-26        169.164702
4  2024-05-27        167.960621
5  2024-05-28        167.837619
6  2024-05-29        167.844915
7  2024-05-30        167.827018
8  2024-05-31        168.257723
9  2024-06-01        169.179291
10 2024-06-

Fetched data for TSLA:
        index        open        high         low       close    adjclose  \
0 2022-05-24  217.843338  217.973328  206.856674  209.386673  209.386673   
1 2022-05-25  207.949997  223.106674  207.669998  219.600006  219.600006   
2 2022-05-26  220.473328  239.556671  217.886673  235.910004  235.910004   
3 2022-05-27  241.083328  253.266663  240.176666  253.210007  253.210007   
4 2022-05-31  257.946655  259.600006  244.743332  252.753326  252.753326   

      volume ticker  
0   89092500   TSLA  
1   92139300   TSLA  
2  106003200   TSLA  
3   89295000   TSLA  
4  101914500   TSLA  
Forecasted Close Prices for TSLA:
           ds  Forecasted Close
0  2024-05-23        180.110001
1  2024-05-24        180.470566
2  2024-05-25        211.097031
3  2024-05-26        211.096755
4  2024-05-27        180.122099
5  2024-05-28        180.794611
6  2024-05-29        180.277092
7  2024-05-30        180.108102
8  2024-05-31        180.468667
9  2024-06-01        211.095133
1

Fetched data for UNH:
        index        open        high         low       close    adjclose  \
0 2022-05-24  490.790009  498.079987  488.660004  497.559998  483.508209   
1 2022-05-25  498.519989  502.309998  491.260010  498.089996  484.023132   
2 2022-05-26  501.739990  507.260010  499.410004  502.230011  488.046326   
3 2022-05-27  491.000000  507.109985  489.390015  507.109985  492.788452   
4 2022-05-31  503.619995  504.109985  495.660004  496.779999  482.750214   

    volume ticker  
0  2658100    UNH  
1  3161600    UNH  
2  2705800    UNH  
3  3349100    UNH  
4  4003100    UNH  
Forecasted Close Prices for UNH:
           ds  Forecasted Close
0  2024-05-23        521.349976
1  2024-05-24        521.761897
2  2024-05-25        510.756953
3  2024-05-26        510.757572
4  2024-05-27        521.574144
5  2024-05-28        521.065275
6  2024-05-29        521.349770
7  2024-05-30        521.354289
8  2024-05-31        521.766210
9  2024-06-01        510.761267
10 2024-06-02  

Fetched data for V:
        index        open        high         low       close    adjclose  \
0 2022-05-24  203.910004  204.669998  199.029999  202.630005  199.488754   
1 2022-05-25  200.589996  204.949997  200.470001  203.839996  200.679962   
2 2022-05-26  205.110001  209.740005  204.669998  208.550003  205.316940   
3 2022-05-27  210.000000  213.229996  209.639999  212.880005  209.579819   
4 2022-05-31  210.380005  214.350006  209.110001  212.169998  208.880829   

    volume ticker  
0  7458500      V  
1  5457200      V  
2  9212300      V  
3  4897200      V  
4  9586400      V  
Forecasted Close Prices for V:
           ds  Forecasted Close
0  2024-05-23        275.579987
1  2024-05-24        275.886834
2  2024-05-25        265.610723
3  2024-05-26        265.614248
4  2024-05-27        275.326416
5  2024-05-28        275.177804
6  2024-05-29        275.443444
7  2024-05-30        275.604668
8  2024-05-31        275.911515
9  2024-06-01        265.635404
10 2024-06-02      

Fetched data for XOM:
        index       open       high        low      close   adjclose    volume  \
0 2022-05-24  92.820000  94.790001  92.320000  94.400002  88.141296  24293300   
1 2022-05-25  94.580002  96.529999  94.470001  96.300003  89.915329  29947800   
2 2022-05-26  96.599998  97.430000  96.199997  96.639999  90.232788  24314800   
3 2022-05-27  95.949997  97.930000  95.300003  97.589996  91.119797  25588800   
4 2022-05-31  98.559998  99.779999  95.709999  96.000000  89.635223  44888100   

  ticker  
0    XOM  
1    XOM  
2    XOM  
3    XOM  
4    XOM  
Forecasted Close Prices for XOM:
           ds  Forecasted Close
0  2024-05-23        115.480003
1  2024-05-24        115.648335
2  2024-05-25        105.701866
3  2024-05-26        105.697153
4  2024-05-27        115.461661
5  2024-05-28        115.570557
6  2024-05-29        115.436500
7  2024-05-30        115.447015
8  2024-05-31        115.615347
9  2024-06-01        105.668878
10 2024-06-02        105.664165
11 2024